*****
## Additional Layer Extract
*****
Author: Mackenzie Rock

Date: June 16, 2025

Goal: The goal of this Jupyter notebook is to extract the additional relevant GeoTIFF layers and store/upload them to the postgreSQL database. In this section I will test:
- Understand the structure of the additiona layers and the requirements for the table schema
- Write the code to extract the data
- I will visualize the data to ensure I have capture it correctly
- Transformation into suitable format for load
- Load them into the postgreSQL database

Soil Data: https://sis.agr.gc.ca/cansis/nsdb/psm/index.html

2020 Land Cover Data: https://open.canada.ca/data/en/dataset/ee1580ab-a23d-4f86-a09b-79763677eb47

#### Layer Selection

I will utilize the following layers in my project.

The following soil data at a depth of 0-5cm
- Bulk density (g/cm^3)
- Cation exchange capacity (meq/100g)
- Soil organic carbon (%)
- pH in CaCl2
- Sand percentage (%)
- Silt percentage (%)
- Clay percentage (%)

There are more options within the soil landscape offering. With 5-15cm, 15-30cm, 30-60cm, and 60-100cm options for all of the above. I'm restricting the scope of this project to 0-5cm as an MVP. Each tiff is nearly 3GB so adding all data represents additional cost in both the database and frontend (required compute).

The following terrain data:
- Landcover 2020 Classification
    - The landcover 2025 dataset has not been released yet.

In [1]:
import os
import requests
import rasterio
import matplotlib.pyplot as plt
import numpy as np

ADD_OUTPUT_DIR = "./additional_layers"
ADD_MAP_OUTPUT_DIR = "./additional_maps"
os.makedirs(ADD_OUTPUT_DIR, exist_ok=True)
os.makedirs(ADD_MAP_OUTPUT_DIR, exist_ok=True)

def inspect_all_tifs(folder_path="./additional_layers"):
    tif_files = [f for f in os.listdir(folder_path) if f.endswith(".tif")] #list of tif files

    if not tif_files:
        print("No .tif files found in the directory.")
        return

    for filename in tif_files:
        file_path = os.path.join(folder_path, filename) #for each file name in tif files join the folder path with the file name to get file path
        print(f"\nFile: {filename}")

        #check the file attributes for each of the .tif files

        print(filename)

        try:
            with rasterio.open(file_path) as src:
                print(f"  - CRS: {src.crs}")
                print(f"  - Width x Height: {src.width} x {src.height}")
                print(f"  - Number of Bands: {src.count}")
                print(f"  - Data Type(s): {src.dtypes}")
                print(f"  - Bounds: {src.bounds}")
                print(f"  - NoData Value: {src.nodata}")
                print(f"  - Transform: {src.transform}")
        except Exception as e:
            print(f"Failed to read {filename}: {e}")

inspect_all_tifs()



File: soc.tif
soc.tif
  - CRS: PROJCS["Lambert_Conformal_Conic_2SP",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Conformal_Conic_2SP"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-95],PARAMETER["standard_parallel_1",49],PARAMETER["standard_parallel_2",77],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
  - Width x Height: 54243 x 45865
  - Number of Bands: 1
  - Data Type(s): ('float32',)
  - Bounds: BoundingBox(left=-2384975.0, bottom=5860375.0, right=3039325.0, top=10446875.0)
  - NoData Value: nan
  - Transform: | 100.00, 0.00,-2384975.00|
| 0.00,-100.00, 10446875.00|
| 0.00, 0.00, 1.00|

File: silt.tif
silt.tif
  - CRS: PROJCS["Lambert_Conformal_Conic_2SP",GEOGCS[

In [3]:
ADD_OUTPUT_DIR = "./additional_layers"
ADD_MAP_OUTPUT_DIR = "./additional_maps"

tif_files = [f for f in os.listdir(ADD_OUTPUT_DIR) if f.endswith(".tif")]

for file in tif_files:
    label = file.split(".")[0]
    label_safe = label.replace(" ", "_")
    tif_path = os.path.join(ADD_OUTPUT_DIR, file)
    with rasterio.open(tif_path) as src:
        scale_factor = 8
        data = src.read(
        1,
        out_shape=(
            int(src.height // scale_factor),
            int(src.width // scale_factor)
        ),
        resampling=rasterio.enums.Resampling.average  # or .bilinear
        )
        transform = src.transform * src.transform.scale(
            (src.width / data.shape[-1]),
            (src.height / data.shape[-2])
        )


        plt.figure(figsize=(10, 6))
        plt.imshow(data, cmap="viridis")
        plt.colorbar(label="Value")
        plt.title(label)
        plt.axis("off")
        img_path = os.path.join(ADD_MAP_OUTPUT_DIR, f"{label_safe}.png")
        plt.savefig(img_path, bbox_inches="tight")
        plt.close()

        print(f"Saved raster visualization to {img_path}")




Saved GeoTIFF to ./additional_layers/soc.tif
Saved raster visualization to ./additional_layers/soc.png
Saved GeoTIFF to ./additional_layers/silt.tif
Saved raster visualization to ./additional_layers/silt.png
Saved GeoTIFF to ./additional_layers/landcover.tif
Saved raster visualization to ./additional_layers/landcover.png
Saved GeoTIFF to ./additional_layers/bulk_density.tif
Saved raster visualization to ./additional_layers/bulk_density.png
Saved GeoTIFF to ./additional_layers/ph.tif
Saved raster visualization to ./additional_layers/ph.png
Saved GeoTIFF to ./additional_layers/clay.tif
Saved raster visualization to ./additional_layers/clay.png
Saved GeoTIFF to ./additional_layers/sand.tif
Saved raster visualization to ./additional_layers/sand.png


In [2]:
import os
import rasterio
import numpy as np
from numpy import float32
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from datetime import date
import pandera as pa
from pandera.typing import DataFrame, Series
from pandera import Field, check_types
from pandera.typing.geopandas import GeoDataFrame
from typing import Dict, Type
import warnings

warnings.filterwarnings('ignore', module='pandera')


# File mappings from logical table name to .tif filename
TABLE_FILE_MAP = {
    "bulk_density_5cm": "bulk_density.tif",
    "clay_5cm": "clay.tif",
    "landcover": "landcover.tif",
    "ph_5cm": "ph.tif",
    "sand_5cm": "sand.tif",
    "silt_5cm": "silt.tif",
    "soc_5cm": "soc.tif"
}

# === Schema and Validator === #
class RasterPointSchema(pa.DataFrameModel):
    lon: Series[float32]
    lat: Series[float32]
    value: Series[float32]
    acquisition_date: Series[date]

class RasterPointValidator:
    @staticmethod
    @check_types
    def validate(df: GeoDataFrame[RasterPointSchema]) -> GeoDataFrame[RasterPointSchema]:
        assert isinstance(df, gpd.GeoDataFrame), "Not a GeoDataFrame"
        return df

VALIDATOR_REGISTRY: Dict[str, Type[RasterPointValidator]] = {
    table: RasterPointValidator for table in TABLE_FILE_MAP.keys()
}

def validate_table(df, table_name):
    validator = VALIDATOR_REGISTRY.get(table_name)
    if validator is None:
        raise ValueError(f"No validator found for table {table_name}")
    return validator.validate(df)

# === Directory containing the .tif files === #
TIF_DIR = "./additional_layers"


In [11]:
def raster_to_downsampled_geodf(file_path: str, acquisition_date: date, scale_factor: int = 30) -> gpd.GeoDataFrame:
    import rasterio
    import numpy as np
    import geopandas as gpd
    from shapely.geometry import Point

    with rasterio.open(file_path) as src:
        # Downsample dimensions
        out_height = src.height // scale_factor
        out_width = src.width // scale_factor

        # Efficient resampled read
        data = src.read(
            1,
            out_shape=(out_height, out_width),
            resampling=rasterio.enums.Resampling.average
        )

        # Adjust transform for new size
        transform = src.transform * src.transform.scale(
            src.width / data.shape[-1],
            src.height / data.shape[-2]
        )

        nodata = src.nodata if src.nodata is not None else -9999
        data = np.where(np.isclose(data, nodata, rtol=1e-5), np.nan, data)
        mask = ~np.isnan(data)
        rows, cols = np.where(mask)
        values = data[rows, cols]

        xs, ys = rasterio.transform.xy(transform, rows, cols, offset='center')
        points = [Point(x, y) for x, y in zip(xs, ys)]

        gdf = gpd.GeoDataFrame({
            "value": values.astype(np.float32),
            "geometry": points,
            "acquisition_date": acquisition_date
        }, crs=src.crs).to_crs(epsg=4326)

        gdf["lon"] = gdf.geometry.x.astype(np.float32)
        gdf["lat"] = gdf.geometry.y.astype(np.float32)

        return gdf

In [12]:
# === Main Preprocessing Loop === #
prepared_dfs = {}

for table_name, filename in TABLE_FILE_MAP.items():
    file_path = os.path.join(TIF_DIR, filename)
    if not os.path.exists(file_path):
        print(f"File not found: {file_path}")
        continue

    try:
        gdf = raster_to_downsampled_geodf(file_path, acquisition_date=date.today(), scale_factor=30)

        # ✅ Step 1: Validate
        validated_gdf = validate_table(gdf, table_name)
        prepared_dfs[table_name] = validated_gdf
        print(f"Validated: {table_name} ({len(validated_gdf)} rows)")

    except Exception as e:
        print(f"Failed to process {filename}: {e}")
        continue

# All validated GeoDataFrames are now in `prepared_dfs` keyed by table name

Validated: bulk_density_5cm (946189 rows)
Validated: clay_5cm (945946 rows)
Validated: landcover (13149238 rows)
Validated: ph_5cm (920495 rows)
Validated: sand_5cm (945946 rows)
Validated: silt_5cm (945946 rows)
Validated: soc_5cm (946244 rows)


In [13]:
display(prepared_dfs['bulk_density_5cm'].head())

,value,geometry,acquisition_date,lon,lat
0,0.739381,POINT (-64.8166 82.91169),2025-06-19,-64.816597,82.911690
1,0.737520,POINT (-64.91086 82.88897),2025-06-19,-64.910858,82.888969
2,0.736772,POINT (-64.72759 82.8773),2025-06-19,-64.727585,82.877304
3,0.809177,POINT (-63.8194 82.81798),2025-06-19,-63.819397,82.817986
4,0.786326,POINT (-63.63939 82.80592),2025-06-19,-63.639393,82.805923


In [15]:
from sqlalchemy import create_engine
from geoalchemy2 import Geometry
from dotenv import load_dotenv

load_dotenv()
engine_url = os.getenv("POSTGRES")

engine = create_engine(engine_url)

# --- Loop through and write each table ---
for table_name, gdf in prepared_dfs.items():
    try:
        gdf.to_postgis(
            name=table_name,
            con=engine,
            if_exists='replace',       # Use 'replace' to overwrite
            index=False,
            dtype={
                "geometry": Geometry(geometry_type="POINT", srid=4326)
            }
        )
        print(f"Inserted: {table_name} ({len(gdf)} rows)")
    except Exception as e:
        print(f"Failed to insert {table_name}: {e}")

Inserted: bulk_density_5cm (946189 rows)
Inserted: clay_5cm (945946 rows)
Inserted: landcover (13149238 rows)
Inserted: ph_5cm (920495 rows)
Inserted: sand_5cm (945946 rows)
Inserted: silt_5cm (945946 rows)
Inserted: soc_5cm (946244 rows)
